In [ ]:
import pandas as pd
import re

In [ ]:
df_injuries = pd.read_csv("injuries_2010-2023early.csv")
df_season_dates = pd.read_csv("season schedule.csv")

In [ ]:
#Create a column with One month before season starts to indicate the start of the season for the players
df_season_dates['Start_Date_Minus_One_Month'] = pd.to_datetime(df_season_dates['start_date']) - pd.DateOffset(months=1)


In [ ]:
df_season_dates["end_date_join"] = df_season_dates["Start_Date_Minus_One_Month"].shift(-1) - pd.DateOffset(days=1)


# replace the last row of end_date_join with the end_date since its empty due to the next season schedule not released yet 
df_season_dates.iloc[-1, df_season_dates.columns.get_loc('end_date_join')] = df_season_dates.iloc[-1, df_season_dates.columns.get_loc('end_date')]


In [ ]:
#remove the acquired column and rows which are empty in Relinquished
if "Acquired" in df_injuries.columns:
    df_injuries = df_injuries.drop(columns="Acquired")
df_injuries2 = df_injuries[df_injuries.Relinquished.notnull()]

In [ ]:
#Replace all team names which are Bobcats with Hornets as they are the same team
df_injuries2["Team"] = df_injuries2["Team"].apply(lambda x: 'Hornets' if x == "Bobcats" else x)
#fill the empty values in Team with no team available
df_injuries2.update(df_injuries2['Team'].fillna(value='No team available', inplace=True))


#Fill a new column in df_injuries with the corresponding season 
df_injuries2["season"] = df_injuries2["Date"].apply(lambda x: df_season_dates.loc[(df_season_dates['Start_Date_Minus_One_Month'] <= x) & (df_season_dates['end_date_join'] >= x), 'season'].values[0])

display(df_injuries2)
display(df_injuries2["season"].unique())

In [ ]:
#edit the names of players to remove the extra dot and select only one name if there is a '/' in the into
df_injuries2["Relinquished"] = df_injuries2["Relinquished"].apply(lambda x: x.replace('•',"").split('/')[-1].strip() if '/' in x else x.replace('•',"").strip())
#remove the brackets and the names in them
df_injuries2["Relinquished"] = df_injuries2["Relinquished"].apply(lambda x: re.sub(r'\(.*\)', "", x).strip())

display(df_injuries2)

In [ ]:
df_injuries2.to_csv("injuries_processed.csv", index=False)

In [ ]:
#after Tableau Prep

In [ ]:
df_players = pd.read_csv("players_data.csv")

In [ ]:
# Identify duplicate rows based on specific columns
duplicate_rows = df_players.duplicated(subset=['player_name', 'season'], keep=False)

# Remove duplicate rows from the dataframe
df_players = df_players[~duplicate_rows]


In [ ]:
df_players.to_csv("players_data_2.csv", index=False)

In [ ]:
#Further proccessing the data output from Tableau Prep
df_tea_prep = pd.read_csv("TPrepped_data.csv")
display(df_tea_prep)

In [ ]:
df_tea_prep['player_name'] = df_tea_prep['Relinquished']
df_tea_prep['season-1'] = df_tea_prep['season']
display(df_tea_prep)

In [ ]:
df_tea_prep2 = df_tea_prep.copy()
df_tea_prep2.sort_values(by=['player_name', 'age', 'player_height', 'player_weight'], inplace=True)
# Group by player_name and forward fill NaN values in each group separately
df_tea_prep2[['age', 'player_height', 'player_weight']] = df_tea_prep2.groupby('player_name')[['age', 'player_height', 'player_weight']].fillna(method='ffill')
df_tea_prep2[['age', 'player_height', 'player_weight']] = df_tea_prep2.groupby('player_name')[['age', 'player_height', 'player_weight']].fillna(method='bfill')
display(df_tea_prep2)

In [ ]:
df_tea_prep2.sort_values(by=['Date'], inplace=True)
display(df_tea_prep2)

In [ ]:
df_tea_prep2.to_csv("Combined.csv", index=False)